In [ ]:
import sys
from tqdm import tqdm

# TO CHANGE
BASEDIR = "../../"
sys.path.insert(0, BASEDIR)

In [ ]:
TRIALS = 2
FIX_FILE_PATH = "./import_fix.py"
for _ in range(TRIALS):
    try:
      from src import PersonalAI, PersonalAIConfig, QAPipelineConfig, MemPipelineConfig, \
            GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig

      from src.db_drivers import KeyValueDriverConfig, GraphDriverConfig, VectorDriverConfig
      from src.db_drivers.kv_driver import DEFAULT_INMEMORYKV_CONFIG
      from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG
      from src.db_drivers.vector_driver import VectorDBConnectionConfig

      from src.pipelines.qa.knowledge_retriever import AStarGraphSearchConfig, AStarMetricsConfig, BFSSearchConfig, MixturedGraphSearchConfig
      from src.pipelines.qa import QueryLLMParserConfig, KnowledgeComparatorConfig, KnowledgeRetrieverConfig, QALLMGeneratorConfig

      from src.pipelines.memorize import LLMExtractorConfig, LLMUpdatorConfig

      from src.utils import NodeType, Logger
    except RuntimeError as e:
        from pathlib import Path
        fix_path = Path(FIX_FILE_PATH)
        if fix_path.is_file():
            %run {fix_path} --base_dir BASEDIR
        else:
            raise e

#### 1. Загружем датасет с текстами, на основе которого будет построен граф знаний

In [ ]:
# Loading dataset to build graph and other structures 
# TODO

DATASET_PATH = ...
data = ...

#### 2. Задаём конфигурацию графа знаний

In [ ]:
# Graph model configuration
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG)

In [ ]:
# Vector model configuration
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/testing'
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/testing'
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

DEVICE = 'cuda'
EMBEDDER_MODEL_PATH = '../../models/intfloat/multilingual-e5-small'
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

In [ ]:
KV_STORAGE_CONFIG = KeyValueDriverConfig(db_vendor='inmemory_kv', db_config=DEFAULT_INMEMORYKV_CONFIG)
ASTAR_RETRIEVER_CONFIG = AStarGraphSearchConfig(
    metrics_config=AStarMetricsConfig(h_metric_name='ip', kvdriver_config=KV_STORAGE_CONFIG),
    max_depth=20, max_passed_nodes=1000,
    accepted_node_types=[NodeType.object , NodeType.hyper, NodeType.episodic])

BFS_RETRIEVER_CONFIG = BFSSearchConfig(
    strict_filter = True, hyper_episodic_num = 15,
    chain_triplets_num = 25, other_triplets_num = 6)


RETRIEVER_NAME = 'mixture'
RETRIEVER_CONFIG = MixturedGraphSearchConfig(
    astar_config=ASTAR_RETRIEVER_CONFIG,
    bfs_config=BFS_RETRIEVER_CONFIG
)

In [ ]:
LANGUAGE = 'en' # TO CHANGE ('ru' | 'en' | 'auto')

In [ ]:
# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    query_parser_config=QueryLLMParserConfig(lang=LANGUAGE),
    knowledge_comparator_config=KnowledgeComparatorConfig(),
    knowledge_retriever_config=KnowledgeRetrieverConfig(
        retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_CONFIG),
    answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE))

# Memorize-pipeline configuration
MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang=LANGUAGE),
    updator_config=LLMUpdatorConfig(lang=LANGUAGE))

PERSONALAI_CONFIG = PersonalAIConfig(
    graph_struct_config=GRAPH_MODEL_CONFIG,
    embedds_struct_config=VECTOR_MODEL_CONFIG,
    qa_pipeline_config=QA_PIPELINE_CONFIG,
    mem_pipeline_config=MEM_PIPELINE_CONFIG,
    log=Logger('log/main'))

#### 3. Инициализируем граф знаний

In [ ]:
personalai = PersonalAI(config=PERSONALAI_CONFIG)

#### 4. Добавляем в граф информацию из загрженного датасета

In [ ]:
messages = [...]
properties = [dict() for _ in range(len(messages))]

triplets, info = [], []
for text, prop in tqdm(list(zip(messages, properties))):
    tmp_triplets, tmp_info = personalai.update_memory(text, prop)
    triplets.append(tmp_triplets)
    info.append(tmp_info)

#### 5. Q&A

In [ ]:
# TODO
examples_questions = []

In [ ]:
answer, info = personalai.answer_question(examples_questions[0])
print(answer)